In [32]:
import pandas as pd

In [71]:
#Find your user_id by logging into your account, going to your profile, going to your workouts, and get the link from "Download Workout"
import requests
from config import user_id, username, password
s = requests.Session()
payload = {'username_or_email':username, 'password':password}
s.post('https://api.onepeloton.com/auth/login', json=payload)
download = f'https://api.onepeloton.com/api/user/{user_id}/workout_history_csv?timezone=America/Chicago'
response = s.get(download)
with open('C:/Users/erdos/Class Folder/Peloton_Data/Resources/input_workouts.csv', 'wb') as file:
    file.write(response.content)

In [72]:
#read the imported CSV file into a dataframe
df = pd.read_csv('Resources/input_workouts.csv')

In [73]:
# convert the column of time objects to datetime objects
df['date'] = df['Workout Timestamp'].str.slice(stop=-12)
df['Workout Timestamp'] = df['date'].apply(pd.to_datetime)
df.drop(['date','Class Timestamp','Avg. Incline','Avg. Pace (min/mi)'], axis=1, inplace=True)
df.fillna(0, inplace=True)

#convert float to int for columns that will always be whole numbers. 
df[['Total Output', 'Avg. Watts', 'Avg. Cadence (RPM)']] = df[['Total Output', 'Avg. Watts', 'Avg. Cadence (RPM)']].astype(int)

#Setup initial FTP score: 
# set the threshold date - change date relevant to your data
threshold_date = pd.to_datetime('2023-04-01')
df['Current FTP'] = 0
# set the value of 'Current FTP' 
df.loc[df['Workout Timestamp'] < threshold_date, 'Current FTP'] = 200

# ----- Uncomment when you want to add a new FTP score-----#
# def add_current_ftp(df, new_ftp, date):
#     # set the value of 'Current FTP' to the input value for the rows where the date is after the input date
#     df.loc[df['Workout Timestamp'] >= date, 'Current FTP'] = new_ftp
    
#     # print the resulting DataFrame
#     print(df)

# #add the date the new FTP score was documented. 
# add_current_ftp(df, <new ftp value>, pd.to_datetime(<date of new ftp>))

#move the columns to a place that makes sense. 
seconds_column = df.pop('Length (seconds)')
df.insert(4, 'Length (seconds)', seconds_column)
ftp_column = df.pop('Current FTP')
df.insert(8, 'Current FTP', ftp_column)

# create a fitness indicactor column by dividing the avg watts by the avg heartrate for Cycling only 
cycling_mask = df['Fitness Discipline'] == 'Cycling'
fitness_indicator = df.loc[cycling_mask, 'Avg. Watts'] / df.loc[cycling_mask, 'Avg. Heartrate']
df.loc[cycling_mask, 'Fitness Indicator'] = fitness_indicator.round(2)

# Create a column that will assign avg power zones for your cycling workout. 
watt_ranges = [(0,0,''), (1, 55, 'Active Recovery'), (56, 75, 'Endurance'), (76, 90, 'Tempo'), (91, 105, 'Threshold'), (106, 120, 'VO2 Max')]

for index, row in df.iterrows():
    # get the current FTP and average watts for this row
    current_ftp = row['Current FTP']
    avg_watts = row['Avg. Watts']
    # calculate the percentage of the FTP score
    ftp_percentage = avg_watts / current_ftp * 100
    # loop through the watt ranges and categorize the average watts
    for watt_range in watt_ranges:
        if watt_range[0] <= ftp_percentage <= watt_range[1]:
            # assign the power zone to the 'Avg. Power Zone' column
            df.at[index, 'Avg. Power Zone'] = watt_range[2]
            break

df.head(5)

,Workout Timestamp,Live/On-Demand,Instructor Name,Length (minutes),Length (seconds),Fitness Discipline,Type,Title,Current FTP,Total Output,Avg. Watts,Avg. Resistance,Avg. Cadence (RPM),Avg. Speed (mph),Distance (mi),Calories Burned,Avg. Heartrate,Fitness Indicator,Avg. Power Zone
0,2023-01-30,On Demand,Ben Alldis,5,300,Cycling,Warm Up/Cool Down,5 min Warm Up Ride,200,41,137,39%,90,17.95,1.49,61,0.00,inf,Endurance
1,2023-01-31,Live,Jenn Sherman,15,900,Cycling,Intervals,15 min HIIT Ride,200,137,153,41%,84,17.95,4.49,256,159.52,0.96,Tempo
2,2023-02-01,Live,Robin Arzon,15,900,Cycling,Music,15 min 80s Ride,200,112,125,41%,84,17.55,4.39,206,137.85,0.91,Endurance
3,2023-02-02,Live,Bradley Rose,30,1800,Cycling,Low Impact,30 min Low Impact Ride,200,150,84,35%,78,14.61,7.30,322,118.81,0.71,Active Recovery
4,2023-02-04,On Demand,Bradley Rose,15,900,Cycling,Climb,15 min Climb Ride,200,158,176,51%,72,19.63,4.91,253,157.94,1.11,Tempo


In [76]:
df.to_csv('Resources/output_workouts.csv')